In [1]:
import pandas as pd
import numpy as np
import folium
# Importing the functions created in kiosk.py
from kiosk import haversinedistance
from kiosk import getLatLong
from kiosk import findShortestDist
from kiosk import shortestRoute_Greedy
from kiosk import calculateDistance
from kiosk import shortestRoute_Greedy
from kiosk import print_route
from kiosk import two_opt_swap
from kiosk import two_opt

%matplotlib inline

In [2]:
# Reading the csv file
df = pd.read_csv("../input/Kiosk-Coords.csv")

In [3]:
# eliminating the duplicate rows - '100 E Wisconsin' and 'Allstate HQ (Tenants Only)'
df =df.drop_duplicates()

In [4]:
df.head()

,name,address (S),latitude (N),longitude (N)
0,University of Illinois at Chicago - Student Ce...,"Student Residence Hall, Chicago, IL 60612",41.871103,-87.674502
1,600 W Chicago,"600 W Chicago Ave, Chicago, IL 60654",41.897519,-87.645072
2,Chase Tower,"Chase Tower, 111 E Wisconsin Ave, Milwaukee, W...",43.037923,-87.909352
3,7-Eleven @ Kingsbury and Ontario,"645 N Kingsbury St, Chicago, IL 60654",41.893390,-87.641096
4,Feinberg Pavilion - Northwestern Medicine,"251 E Huron St, Chicago, IL 60611",41.894640,-87.621128


In [5]:
# Add the Start and Destination location to the data frame and indexing it to 0th position.
df.loc[-1] = ["Farmers Fridge","Lake and Racine Ave","41.8851024","-87.6618988"]
df.index = df.index + 1
df = df.sort_index()

In [6]:
df.head()

,name,address (S),latitude (N),longitude (N)
0,Farmers Fridge,Lake and Racine Ave,41.8851024,-87.6618988
1,University of Illinois at Chicago - Student Ce...,"Student Residence Hall, Chicago, IL 60612",41.8711,-87.6745
2,600 W Chicago,"600 W Chicago Ave, Chicago, IL 60654",41.8975,-87.6451
3,Chase Tower,"Chase Tower, 111 E Wisconsin Ave, Milwaukee, W...",43.0379,-87.9094
4,7-Eleven @ Kingsbury and Ontario,"645 N Kingsbury St, Chicago, IL 60654",41.8934,-87.6411


I have used the haversine formula to find the distance between two coordinates given by lattitude and longitude
Refered links:<br>
https://en.wikipedia.org/wiki/Haversine_formula
<br>
https://www.movable-type.co.uk/scripts/latlong.html
<br>
https://www.researchgate.net/publication/282314345_Landmark_based_shortest_path_detection_by_using_Dijkestra_Algorithm_and_Haversine_Formula

Haversine formula
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/47a496cca1b6d57e0ae7b462c1678660392d1057" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.505ex; width:57.49ex; height:6.176ex;" alt="{\displaystyle \operatorname {hav} \left({\frac {d}{r}}\right)=\operatorname {hav} (\varphi _{2}-\varphi _{1})+\cos(\varphi _{1})\cos(\varphi _{2})\operatorname {hav} (\lambda _{2}-\lambda _{1})}">

where

<li>hav is the haversine function:
The haversine, in particular, was important in navigation because it appears in the haversine formula, which is used to accurately compute distances within reason on an astronomic spheroid (see issues with the earth's radius vs. sphere) given angular positions (e.g., longitude and latitude).

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/7f6323f8e404496c8253539689327f5228699935" class="mwe-math-fallback-image-inline" aria-hidden="true" style="vertical-align: -2.505ex; width:32.959ex; height:6.343ex;" alt="\operatorname {hav} (\theta )=\sin ^{2}\left({\frac {\theta }{2}}\right)={\frac {1-\cos(\theta )}{2}}">
<li>d is the distance between the two points (along a great circle of the sphere; see spherical distance),
<li>r is the radius of the sphere,
<li>φ1, φ2: latitude of point 1 and latitude of point 2, in radians
<li>λ1, λ2: longitude of point 1 and longitude of point 2, in radians

In [7]:
# Checking the haversine distance function
l1 = 41.8851024
l2 = -87.6618988
l3 = 42.196
l4 = -88.1733
hd =haversinedistance(l1,l2,l3,l4)
print(hd,"km")

54.576877399 km


In [8]:
# Creating a new dataframe that will just have the name, latitude and longitude.
df_latlon = df.drop('address (S)',axis =1)

In [9]:
# Creating distance matrix with name as index and columns filled with nan values
df_mat = pd.DataFrame(np.nan, index=df_latlon.name, columns=df_latlon.name)

In [10]:
# Using getLatLong function from kiosk.py to find the latitude and longitude for a name value. 
# eg: ''Allstate HQ (Tenants Only)'
lat,long = getLatLong(df_latlon,'Allstate HQ (Tenants Only)')
print(lat)
print(long)

42.096740000000004
-87.87009499999999


In [11]:
# Filling distance matrix with the distances(km) using haversine distance function
# to get distance between every kiosk co-ordinate.
for ind in df_mat.index:
    lat1,lon1 = getLatLong(df_latlon,ind)
    for col in df_mat.columns:
        #print(col)
        lat2,lon2 = getLatLong(df_latlon,col)
        #print(lat1,lon1,lat2,lon2)
        hav_dist = haversinedistance(lat1,lon1,lat2,lon2)
        df_mat[ind][col] = hav_dist     

In [12]:
df_mat

name,Farmers Fridge,University of Illinois at Chicago - Student Center West,600 W Chicago,Chase Tower,7-Eleven @ Kingsbury and Ontario,Feinberg Pavilion - Northwestern Medicine,Chicago Midway Airport - Ticketing Employee Lounge,DeVry Chicago Campus (Students/Staff Only),525 W Monroe,200 W Jackson (Tenants Only),...,Skokie Hospital / Northshore,DeVry Addison (Students/Staff Only),CVS @ 344 Hubbard,REI Building,Peggy Notebaert Nature Museum,MillerCoors HQ,Good Samaritan Hospital,Moraine Valley Community College: Police Academy- Building B,O'Hare Terminal 2 - Gate F6,Good Shepherd Hospital
name,,,,,,,,,,,,,,,,,,,,,
Farmers Fridge,0.000000,1.874012,1.961125,129.784899,1.953080,3.537629,12.633518,6.796841,1.883904,2.401319,...,20.191497,31.981525,2.119563,2.761561,5.060180,2.168857,2.008345,2.008345,22.494800,54.581710
University of Illinois at Chicago - Student Center West,1.874012,0.000000,3.816157,131.167134,3.713455,5.135567,10.761119,8.021792,3.020878,3.430813,...,21.366805,31.367485,3.745691,4.617377,6.912548,3.184911,3.847841,3.847841,22.323623,54.797556
600 W Chicago,1.961125,3.816157,0.000000,128.646467,0.564841,2.007569,14.546947,6.272190,1.969632,2.294705,...,19.431527,33.066238,1.041981,1.188400,3.316452,2.222111,0.142835,0.142835,23.208447,54.813625
Chase Tower,129.784899,131.167134,128.646467,0.000000,129.154263,129.306667,139.624950,123.149115,130.614388,130.880588,...,109.989156,121.460824,129.559122,127.461930,125.549254,130.851258,128.733513,128.733513,118.279915,96.072944
7-Eleven @ Kingsbury and Ontario,1.953080,3.713455,0.564841,129.154263,0.000000,1.658671,14.355247,6.836692,1.469647,1.748145,...,19.986179,33.474094,0.478754,1.726575,3.726499,1.697226,0.441565,0.441565,23.685647,55.354277
Feinberg Pavilion - Northwestern Medicine,3.537629,5.135567,2.007569,129.306667,1.658671,0.000000,15.481762,7.858286,2.247663,2.097962,...,20.614546,35.073468,1.421168,2.708494,3.819186,2.248843,1.872872,1.872872,25.172329,56.589555
Chicago Midway Airport - Ticketing Employee Lounge,12.633518,10.761119,14.546947,139.624950,14.355247,15.481762,0.000000,17.601348,13.240869,13.413744,...,29.804727,30.424713,14.260233,15.372149,17.658458,13.238497,14.561483,14.561483,24.735655,57.677559
DeVry Chicago Campus (Students/Staff Only),6.796841,8.021792,6.272190,123.149115,6.836692,7.858286,17.601348,0.000000,8.092241,8.511047,...,13.399275,28.716619,7.314070,5.203993,4.788328,8.385836,6.402564,6.402564,18.096651,48.777099
525 W Monroe,1.883904,3.020878,1.969632,130.614388,1.469647,2.247663,13.240869,8.092241,0.000000,0.518557,...,21.358283,33.861684,1.137354,3.157489,5.186140,0.307598,1.881147,1.881147,24.355452,56.325111


In [13]:
df_mat.to_csv("../output/Haversine_Distance.csv")

In [14]:
# Get the names of the locations
names = df_mat.index

In [15]:
# get the positions of the locations into the location_ind list
location_ind = []
for i in range(0,names.size):
    location_ind.append(i)

In [16]:
# Creating a two dimension numpy array from distance matrix
distarray = np.array(df_mat)

In [17]:
distarray.shape

(49, 49)

In [18]:
def map_route(route,color,filename):
    """
    The function for visualizing the route on the Map.
    Parameters:
    route : is the list of location indices that is to be followed to build a path.
    color: color code to be used to display the path.
    filename: name of the file(html) which is used to display the path
    Returns an folium.Map object.
    """
    mapit = folium.Map([41.8781, -87.6298], zoom_start=11)
    points = []
    for i in route:
        row = df.iloc[i]
        address= row['address (S)']
        #print((name))
        c1 = float(row['latitude (N)'])
        c2 = float(row['longitude (N)'])
        popup = folium.Popup(address, parse_html=True)
        points.append((c1,c2))
        if i == 0:
            # The starting of the path is represented in red color
            folium.Marker(location=[c1,c2],popup=popup, icon=folium.Icon(color='red', icon_color='white', icon='male', angle=0, prefix='fa')).add_to( mapit )
            #print(c1,c2)
        else:
            folium.Marker(location=[c1,c2],popup=popup, icon=folium.Icon(color='darkblue', icon_color='white', icon='male', angle=0, prefix='fa')).add_to( mapit )

    folium.PolyLine(points, color=color, weight=2.5, opacity=1).add_to(mapit)
    #print(points)
    mapit.save("../output/"+filename)
    return mapit

<h1> Using Greedy approach</h1>

In [19]:
greedy_route,totaldistance,dList = shortestRoute_Greedy(distarray,location_ind)

In [20]:
print_route(greedy_route,totaldistance,names)

The route to be followed :
Route   ->  Farmers Fridge  ->  University of Illinois at Chicago - Behavioral Sciences  ->  University of Illinois at Chicago - Richard J Daley Library (Students/Staff Only)  ->  7-Eleven @ Jackson and Desplaines  ->  Epic Burger West Loop  ->  525 W Monroe  ->  CME Center  ->  General Growth Properties HQ (Employees Only)  ->  Merchandise Mart  ->  CVS @ 344 Hubbard  ->  7-Eleven @ Kingsbury and Ontario   ->  North Park University  ->  Good Samaritan Hospital  ->  Medical College of Wisconsin  ->  600 W Chicago  ->  Moraine Valley Community College: Police Academy- Building B  ->  REI Building   ->  DePaul University - Schmitt Academic Center  ->  Peggy Notebaert Nature Museum  ->  Prentice Women`s Hospital - Northwestern Medicine  ->  Feinberg Pavilion - Northwestern Medicine  ->  Illinois Center  ->  CVS @ 137 State  ->  CNA Center (Employees Only)  ->  200 W Jackson (Tenants Only)  ->  311 S Wacker (Tenants Only)  ->  MillerCoors HQ  ->  University of Il

In [21]:
# Displaying Greedy route on map
greedy = map_route(greedy_route,"red","Greedy.html")

In [22]:
greedy
# The red color icon indicates the start and the end. ( Lake and Racine Ave)

<h1>Using 2opt Swap approach </h1>

In [23]:
# Passing the greedy route to the two_opt function to optimize  the distance.
bestroute1, bestdist, bestroute2, bestdist2 = two_opt(greedy_route,distarray)

<h2> Route for Driver 1</h2>

In [24]:
# Best route for driver one according to two_opt Swap algorithm
# The best route
print_route(bestroute1,bestdist,names)

The route to be followed :
Route   ->  Farmers Fridge  ->  University of Illinois at Chicago - Behavioral Sciences  ->  University of Illinois at Chicago - Richard J Daley Library (Students/Staff Only)  ->  7-Eleven @ Jackson and Desplaines  ->  Epic Burger West Loop  ->  525 W Monroe  ->  MillerCoors HQ  ->  311 S Wacker (Tenants Only)  ->  200 W Jackson (Tenants Only)  ->  CNA Center (Employees Only)  ->  CVS @ 137 State  ->  Illinois Center  ->  Feinberg Pavilion - Northwestern Medicine  ->  Prentice Women`s Hospital - Northwestern Medicine  ->  CME Center  ->  General Growth Properties HQ (Employees Only)  ->  Merchandise Mart  ->  CVS @ 344 Hubbard  ->  7-Eleven @ Kingsbury and Ontario   ->  North Park University  ->  Good Samaritan Hospital  ->  Medical College of Wisconsin  ->  Moraine Valley Community College: Police Academy- Building B  ->  600 W Chicago  ->  REI Building   ->  Peggy Notebaert Nature Museum  ->  DePaul University - Schmitt Academic Center  ->  DeVry Chicago Ca

In [25]:
driver1 = map_route(bestroute1,"green","driver1.html")

In [26]:
driver1
# The red color icon indicates the start and the end. ( Lake and Racine Ave)

<h2> Route for Driver2 </h2>

In [27]:
# The second best path for driver two according to 2opt
print_route(bestroute2,bestdist2,names)

The route to be followed :
Route   ->  Farmers Fridge  ->  University of Illinois at Chicago - Behavioral Sciences  ->  University of Illinois at Chicago - Richard J Daley Library (Students/Staff Only)  ->  7-Eleven @ Jackson and Desplaines  ->  Epic Burger West Loop  ->  525 W Monroe  ->  MillerCoors HQ  ->  311 S Wacker (Tenants Only)  ->  200 W Jackson (Tenants Only)  ->  CNA Center (Employees Only)  ->  CVS @ 137 State  ->  Illinois Center  ->  Feinberg Pavilion - Northwestern Medicine  ->  Prentice Women`s Hospital - Northwestern Medicine  ->  CME Center  ->  General Growth Properties HQ (Employees Only)  ->  Merchandise Mart  ->  CVS @ 344 Hubbard  ->  7-Eleven @ Kingsbury and Ontario   ->  North Park University  ->  Good Samaritan Hospital  ->  Medical College of Wisconsin  ->  Moraine Valley Community College: Police Academy- Building B  ->  600 W Chicago  ->  REI Building   ->  Peggy Notebaert Nature Museum  ->  DePaul University - Schmitt Academic Center  ->  DeVry Chicago Ca

In [28]:
driver2 = map_route(bestroute2,"blue","driver2.html")

In [29]:
driver2
# The red color icon indicates the start and the end. ( Lake and Racine Ave)

<h2>Explanation</h2>
<p>
<ul>
<li> I used haversine distance formula to find the distance in km between two kiosk-coordinates.
<li> <b> Greedy route </b> This route is represented with <b><font color="red">red</font></b> path in the map. The algorithm creates the route by choosing the minimum distance to the next kiosk co-ordinate. If the kiosk co-ordinate is already in the list of visited co-ordinate, we choose the next minimum distance. Total distance = 418 km
<li><b>2-Opt swap routes</b>
    <ul type="square">
    <li>This approach takes the greedy route as input and returns two paths.
    <li> The first route for driver 1, shown in the driver1.html with <b><font color="green">green </font></b> route. The total distance cover is around 343 km. This route is the best and it has no crossover path. Hence this route has minimum distance
    <li> The next route which is suboptimal to the best route is shown in driver2.html with <b><font color="blue">blue</font></b> route. It covers the same distance as that by our greedy algorithm (around 418 km) but has few crossovers compare to the greedy route. So this route can be given to the driver 2 instead of the greedy route
    </ul>
</ul>
<p>